# ARIMA
- Produces Crowd Predicitions based on weather and user selected location

### Load the Libaries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from pmdarima import auto_arima
import pickle
import os
from datetime import datetime,timedelta,date
import holidays as hl

import openmeteo_requests
import requests_cache
from retry_requests import retry

str_trim_date = pd.to_datetime('2025-01-01').to_datetime64()# The start of the dataset
end_trim_date = pd.to_datetime('2025-09-30').to_datetime64()# The end of the dataset, shouldn't assume both actual end at this date for each location

### Load the data set ensure both the dataframe date range and date are in correct format for ARIMA model

In [194]:
Auck_peds = pd.read_csv("data_weather/Final/Auckland_Pedestrian_Hourly.csv")
Dub_peds = pd.read_csv("data_weather/Final/Dublin_Pedestrian_Hourly.csv")

df = pd.concat([Auck_peds, Dub_peds],ignore_index=True)

df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x).to_datetime64())
df = df.sort_values(['Location_ID','Date'])

In [195]:
[df['Date'].values]

[array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
        '2021-01-03T00:00:00.000000000', ...,
        '2025-09-28T00:00:00.000000000', '2025-09-29T00:00:00.000000000',
        '2025-09-30T00:00:00.000000000'], dtype='datetime64[ns]')]

In [196]:
[end_trim_date],[str_trim_date]

([numpy.datetime64('2025-09-30T00:00:00.000000000')],
 [numpy.datetime64('2025-01-01T00:00:00.000000000')])

### Main ARIMA model
- Model creation
- Data splitting
- Fitting model
- Creates pickel files for each location
    - Need seperate pickel files for forecasting each location 
- ARMIA needs to have even spacing between dates
    if gap then a fill in needs to be done for Y values(Dep Var) & Exog(or X Ind Vars)

In [218]:
models = {}
for loc in df['Location_ID'].unique():
    print(loc)
    sub = df[df['Location_ID'] == loc].set_index('Date') # Important for the ARIMA model to function 
    
    y = sub['Avg_Daily_Pedestrian_Count'].asfreq('D').interpolate(method='linear') # D is daily, rate of change fill in
    x = sub[['Holiday',
                'Weather_Temperature_Avg',
                'Weather_Wind_Speed_Avg',
                'Weather_Precipitation_Sum',
                'Weather_Relative_Humidity_Avg']].asfreq('D').interpolate(method='linear') # numeric only
    
    y = y[(y.index <= end_trim_date) & (y.index >= str_trim_date)] # will change depending on new datasets in the future
    x = x[(x.index <= end_trim_date) & (x.index >= str_trim_date)] # will change depending on new datasets in the future 
    print([y.shape,x.shape])

    split = int(len(y) * 0.7) # keep chrno order 

    y_train = y.iloc[:split]
    y_test  = y.iloc[split:]

    # sclar = MinMaxScaler()

    # x_train = sclar.fit_transform(x.iloc[:split])
    # x_test = sclar.transform(x.iloc[split:])

    x_train = x.iloc[:split]
    x_test  = x.iloc[split:]

    print([y_train.shape,y_test.shape,x_train.shape,x_test.shape])

    # Auto-tune ARIMA parameters
    stepwise = auto_arima(y=y_train, # Dep
                          X=x_train, # Indep
                          seasonal=True,
                          m=7, # weekly pattern
                          trace=False,
                          error_action='ignore',
                          suppress_warnings=True)
    
    # Fit SARIMA model
    model = SARIMAX(endog=y_train, # Dep
                    exog=x_train, # Indep
                    order=stepwise.order,
                    seasonal_order=stepwise.seasonal_order,
                    enforce_stationarity=False, # Variance and trends aren't constant set to false
                    enforce_invertibility=False)
    results = model.fit(disp=False)

    day = 70

    y_pred = results.predict(
        start=y_test[:day].index[0],
        end=y_test[:day].index[-1],
        exog=x_test[:day]
    )
    
    models[f'{loc}'] = results

IRDUB_1
[(249,), (249, 5)]
[(174,), (75,), (174, 5), (75, 5)]
IRDUB_3
[(249,), (249, 5)]
[(174,), (75,), (174, 5), (75, 5)]
IRDUB_4
[(249,), (249, 5)]
[(174,), (75,), (174, 5), (75, 5)]
IRDUB_5
[(249,), (249, 5)]
[(174,), (75,), (174, 5), (75, 5)]
NZAUK_1
[(273,), (273, 5)]
[(191,), (82,), (191, 5), (82, 5)]


C:\Users\athar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


NZAUK_2
[(273,), (273, 5)]
[(191,), (82,), (191, 5), (82, 5)]


C:\Users\athar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


NZAUK_4
[(273,), (273, 5)]
[(191,), (82,), (191, 5), (82, 5)]


C:\Users\athar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


NZAUK_5
[(273,), (273, 5)]
[(191,), (82,), (191, 5), (82, 5)]


C:\Users\athar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


### Save model pickel files

In [199]:
os.makedirs("arima_models", exist_ok=True) 
for loc,results in models.items():
    # Save model
    model_path = f"arima_models/{loc}_arima.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(results, f)

#### Use a test case

In [200]:
# Setup the Open-Meteo API client with cache and retry on error # <--- this is from Open Meteo Api Docs
cache_session = requests_cache.CachedSession('.amriacache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [201]:
def Weather_Requester(lat:float,long:float) -> pd.DataFrame:
    url = "https://archive-api.open-meteo.com/v1/archive" # Histroical Data
    params = {
        "latitude": lat,
        "longitude": long,
        "start_date": '2025-09-30',
        "end_date": (date.today()-timedelta(days=1)).strftime('%Y-%m-%d'),
        "daily": ["precipitation_sum", "temperature_2m_mean", "relative_humidity_2m_mean", "wind_gusts_10m_mean"],
        "timezone": "America/New_York",
    }
    responses = openmeteo.weather_api(url, params=params)
    # Basically getting the data for the beginning of the trim point of Sep 30 2025 of the dataset to 1 day - current day   
    dly = responses[0].Daily()

    P1 = dly.Variables(0).ValuesAsNumpy() # Np array's 
    T1 = dly.Variables(1).ValuesAsNumpy()
    R1 = dly.Variables(2).ValuesAsNumpy()
    W1 = dly.Variables(3).ValuesAsNumpy()

    url = "https://seasonal-api.open-meteo.com/v1/seasonal" # Future Data
    params = {
        "latitude": lat,
        "longitude": long,
        "forecast_days": 180,
        "timezone": "America/New_York",
        "daily": ["temperature_2m_mean", "wind_speed_10m_mean", "precipitation_sum", "relative_humidity_2m_mean"]
    }
    responses = openmeteo.weather_api(url, params=params)
    dly = responses[0].Daily()

    T2 = dly.Variables(0).ValuesAsNumpy()
    W2 = dly.Variables(1).ValuesAsNumpy()
    P2 = dly.Variables(2).ValuesAsNumpy()
    R2 = dly.Variables(3).ValuesAsNumpy()

    T = np.concatenate((T1,T2))
    w = np.concatenate((W1,W2))
    P = np.concatenate((P1,P2))
    R = np.concatenate((R1,R2))
    
    # Build the final indep array, holiday and time will be added later
    vstk = pd.DataFrame(data = np.vstack((T,w,P,R)).T,
                        columns=['Weather_Temperature_Avg',
                                 'Weather_Wind_Speed_Avg',
                                 'Weather_Precipitation_Sum',
                                 'Weather_Relative_Humidity_Avg'])

    return vstk

In [202]:
def Holidayer(df:pd.DataFrame,CCode:str) -> pd.DataFrame:
    # Uses country code and each data to find holiday or not
    df['Holiday'] = df['Date'].apply(lambda x: 1 if hl.country_holidays(country=CCode).get(x) != None else 0)
    return df

In [213]:
loc = "NZAUK_1" # This was a location to be displayed to user
with open(f"arima_models/{loc}_arima.pkl", "rb") as f:
    model = pickle.load(f) # grab right pickel file

d = datetime(2025,11,29).date()# User specfiies a date -- test
w = Weather_Requester(-36.8485,174.7633) # Grab weather from past and for future
w.insert(0,'Holiday',0)# Inserting these columns to match indep input
w.insert(0,'Date',range(len(w))) # Use range to fill in date indexing numbers 
# Add in the date range from trim point 2025-09-30
w['Date'] = w['Date'].apply(lambda x: datetime(2025,9,30).date() + timedelta(days=x))
h = Holidayer(w,'IE') # Add in the holiday data
h = h.set_index('Date').asfreq('D').interpolate(method='linear') # numeric only
# Send to predict next set of days
pred_mean = pd.DataFrame(model.get_forecast(exog=h,steps=len(h)).predicted_mean)
print(pred_mean)

            predicted_mean
2025-07-11     6313.527974
2025-07-12     5631.937142
2025-07-13     3952.415313
2025-07-14     5321.695013
2025-07-15     5661.107422
...                    ...
2026-03-14     9027.422004
2026-03-15     6855.277517
2026-03-16     8970.497962
2026-03-17     9476.668836
2026-03-18     9784.316034

[251 rows x 1 columns]


In [214]:
pred_meancp = pred_mean.copy()
idx = pred_meancp.index[:1][0] # deaaling with timestamps
idx

Timestamp('2025-07-11 00:00:00')

In [215]:
f = pd.Timestamp(d) # convert user specified date
f

Timestamp('2025-11-29 00:00:00')

In [216]:
# how to get the forecasted crowd number at a date for a lcoation
[pred_meancp['predicted_mean'].loc[f]] 

[8784.321120894465]

In [217]:
print(*pred_meancp['predicted_mean'].loc[f - timedelta(days=30):f + timedelta(days=10)].items(),sep='\n')

(Timestamp('2025-10-30 00:00:00'), 8673.199239607291)
(Timestamp('2025-10-31 00:00:00'), 9248.587278072411)
(Timestamp('2025-11-01 00:00:00'), 8034.03073101135)
(Timestamp('2025-11-02 00:00:00'), 6457.493610934678)
(Timestamp('2025-11-03 00:00:00'), 8026.887653059958)
(Timestamp('2025-11-04 00:00:00'), 8478.44526924403)
(Timestamp('2025-11-05 00:00:00'), 8805.872951263025)
(Timestamp('2025-11-06 00:00:00'), 8476.157840438196)
(Timestamp('2025-11-07 00:00:00'), 8853.743776221445)
(Timestamp('2025-11-08 00:00:00'), 8081.045105048558)
(Timestamp('2025-11-09 00:00:00'), 6404.394135615088)
(Timestamp('2025-11-10 00:00:00'), 8244.583391882095)
(Timestamp('2025-11-11 00:00:00'), 8684.17733286873)
(Timestamp('2025-11-12 00:00:00'), 9069.288528674771)
(Timestamp('2025-11-13 00:00:00'), 7123.70280773749)
(Timestamp('2025-11-14 00:00:00'), 8931.54895080841)
(Timestamp('2025-11-15 00:00:00'), 8230.966023380839)
(Timestamp('2025-11-16 00:00:00'), 6624.710422696017)
(Timestamp('2025-11-17 00:00:00')